In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.abspath(os.path.join(currentdir, os.pardir))
root = os.path.dirname(parent_dir)
print("root Folder Absoloute path: ", root)

import sys
sys.path.append(root)

import logging

logging.basicConfig(level=logging.INFO)

from IPython.display import Image

root Folder Absoloute path:  /home/hfaghihi/Framework/fix/DomiKnowS


In [3]:
import json
import torch

with open("data/train_propara_roberta_version_trips.json", "r") as file:
    data = json.load(file)
print(len(data))
updated_data = []
lines = 0
for item in data:
    lines += len(item["sentence_texts"])
    try:
        temp = item.copy()
        new_state = []
        location_types = []
        for state_id, states in enumerate(temp['states']):
            temp2 = ["-", ]
            
            ltemp = []
            if states[0] == "?":
                ltemp.append("Unknown")
            elif states[0] == "-":
                ltemp.append("-")
            else:
                ltemp.append("Known")
                
            prev_loc = states[0]
            for loc in states[1:]:
                if loc == "?":
                    ltemp.append("Unknown")
                elif loc == "-":
                    ltemp.append("-")
                else:
                    ltemp.append("Known")

                if loc == prev_loc:
                    temp2.append("No change")
                elif loc != "-" and prev_loc == "-":
                    temp2.append("Create")
                elif loc == "-" and prev_loc != "-":
                    temp2.append("Destroy")
                else:
                    temp2.append("Move")
                prev_loc = loc
            new_state.append(temp2)
            location_types.append(ltemp)
        temp['actions'] = new_state
        temp['ltype'] = location_types
        

#         new_annotation = []
#         for state in temp['states_annotation']:
#             temp2 = state
#             for loc in temp2:
#                 if loc[0] == "?":
#                     loc[2] = item['boundaries'][-1][1] + 1
#                     loc[3] = item['boundaries'][-1][1] + 1
#             new_annotation.append(temp2)

#         temp['new_annotation'] = new_annotation
    except:
        print(item)
        raise
    updated_data.append(temp)
    
for item in updated_data:
    steps = len(item['sentence_paragraph'])
    entities = len(item['participants'])
    probs = torch.softmax(torch.randn(steps, entities, 4), dim=-1)
    item['action_probs'] = probs.tolist()
    tprobs = torch.softmax(torch.randn(steps, entities, 4), dim=-1)
    item['Taction_probs'] = tprobs.tolist()
    
with open("data/train.json", "w") as file:
    json.dump(updated_data, file)

391


In [4]:
with open("data/train.json", "r") as file:
    data = json.load(file)
print(len(data))

391


In [5]:
import os
from regr.data.reader import RegrReader
import torch


class ProparaReader(RegrReader):
    def getProcedureIDval(self, item):
        return [item["para_id"]]

    def getEntitiesval(self, item):
        return ' '.join(item['participants'])
        
    def getEntityval(self, item):
        return item["participants"]
    
    def getContextval(self, item):
        return item['sentence_paragraph']

    def getStepval(self, item):
        sentences = item["sentence_texts"]
        return  sentences
    
    def getActionval(self, item):
        actions = []
        for step, step_text in enumerate(item['sentence_texts']):
            actions.append([])
            for eid, entity in enumerate(item['participants']):
                actions[-1].append(item['action_probs'][eid])

        return torch.tensor(actions)
    
    def getTActionval(self, item):
        actions = []
        for step, step_text in enumerate(item['sentence_texts']):
            actions.append([])
            for eid, entity in enumerate(item['participants']):
                actions[-1].append(item['Taction_probs'])

        return torch.tensor(actions)


In [6]:
train_reader = ProparaReader("data/train.json")
# next(iter(train_reader))

In [8]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL, nandL, V, exactL

Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    procedure = Concept("procedure")
    
    context = Concept("context")
    entities = Concept("entities")
    
    (procedure_context, procedure_entities) = procedure.has_a(context, entities)
    
    entity = Concept('entity')
    (entity_rel, ) = entities.contains(entity)
    
    step = Concept("step")
    (context_step, ) = context.contains(step)
    
    before = Concept("before")
    (before_arg1, before_arg2) = before.has_a(arg1=step, arg2=step)
    
    action = Concept(name='action')
    (action_step, action_entity) = action.has_a(step, entity)
    
    create = action(name="create")
    destroy = action(name="destroy")
    move = action(name="move")
    nochange = action('none')
    
    Tcreate = action(name="trips_create")
    Tdestroy = action(name="trips_destroy")
    Tmove = action(name="trips_move")
    Tnochange = action('trips_none')

    exactL(Tcreate, Tdestroy, Tmove, Tnochange)
    exactL(create, destroy, move, nochange)
    
    ifL(Tcreate, create)
    ifL(Tdestroy, destroy)
    ifL(Tmove, move)
    ifL(Tnochange, nochange)
    
#     ifL(
#         destroy('x'), 
#         notL(
#             andL(
#                 existsL(
#                     andL(destroy('y'), existsL(before('y', path=('x', arg1, 'before', arg1)))),
#                     notL(
#                         existsL(
#                             andL(
#                                 create('z'), 
#                                 eqL(before('z'))
#                             )
#                         )
#                     )
#                 )
#             )
#         )
#     )
#   atMostL(1, ("x"), andL(entity, "y", create()))
    # No entity_step




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'step'), ('arg2', 'step')]) is used.


In [ ]:
from regr.sensor.pytorch.sensors import FunctionalSensor, JointSensor, ReaderSensor, FunctionalReaderSensor, JointFunctionalReaderSensor
from regr.sensor.pytorch.learners import ModuleLearner
from regr.sensor.pytorch.relation_sensors import CompositionCandidateSensor
from regr.sensor.pytorch.query_sensor import DataNodeReaderSensor

procedure['id'] = ReaderSensor(keyword='ProcedureID')
context['text'] = ReaderSensor(keyword='Context')
entities['text'] = ReaderSensor(keyword='Entities')

def make_procedure(arg1m, arg2m, data):
    total_procedures = len(arg1m) * len(arg2m)
    rel_links1 = torch.zeros(total_procedures, len(arg1m))
    rel_links2 = torch.zeros(total_procedures, len(arg2m))
    past1 = 0
    past2 = 0
    for i in range(total_procedures):
        rel_links1[i, past2: past2 + len(arg2m)] = 1
        past2 = past2 + len(arg2m)
        rel_links2[i, past1: past1 + len(arg1m)] = 1
        past1 = past1 + len(arg1m)
    
    return rel_links1, rel_links2, data
    
    
procedure[procedure_context.reversed, procedure_entities.reversed, "id"] = JointFunctionalReaderSensor(context['text'], entities['text'], keyword="ProcedureID", forward=make_procedure)

def read_initials(*prev, data):
    number = len(data)
    rel_links = torch.ones(number, 1)
    indexes = [i for i in range(number)]
    return rel_links, data, indexes

entity[entity_rel, 'text', 'index'] = JointFunctionalReaderSensor(entities['text'], keyword='Entity', forward=read_initials)

step[context_step, 'text', 'index'] = JointFunctionalReaderSensor(context['text'], keyword='Step', forward=read_initials)

def make_actions(steps, entities):
    all_actions = len(steps) * len(entities)
    link1 = torch.zeros(all_actions, len(steps))
    link2 = torch.zeros(all_actions, len(entities))
    for i in range(all_actions):
        link1[i: steps[i] * len(entities): (steps[i] + 1) * len(entities)] = 1
        link2[i: entities[i] * len(steps): (entities[i] + 1) * len(steps)] = 1
    
    return link1, link2
    
action[action_step.reversed, action_entity.reversed] = FunctionalSensor(step['index'], entity['index'], forward=make_actions)

# def read_labels(arg1, arg2, data):
    
# action[nochange, destroy, create, move] = FunctionalReaderSensor(action_step.reversed, action_entity.reversed, keyword="Action", forward=read_labels)
